<img  align="right" width="250" src="recycling centre - pic.JPG" style= "float: right; margin: 0 50px 50px 50px;">

# Booking system project 

## Discovery of data insights to evaluate customer patterns and potential cost savings on a recycling centre

***


### Summary: In this project, booking system dataset from a recycling centre located in West London is analysed and main KPIs are calculated. 
### - Data cleaning and transformation was done as data quality issues were identified. 
### - All personal/sensitive data from datasets was encrypted or deleted for GDPR compliance.

<hr style="margin-bottom: 40px;">

### Main insights:


### Import of libraries

In [1]:
import numpy as np 
import pandas as pd
import random
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#dashboard design
import chart_studio.dashboard_objs as dashboard
import IPython.display
from IPython.display import Image


import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from matplotlib.patches import ConnectionPatch

import dash
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (14,8)

from datetime import datetime
import ipywidgets as widgets

In [50]:
import requests
from io import StringIO

#URL of the the raw CSV files from GitHub

url="https://raw.githubusercontent.com/TiagoAnalyst/Professional_Portfolio/main/Booking%20system_dashboard/Data%20files/Recycling%20centre%20A%20-%20bookings.csv"
url2="https://raw.githubusercontent.com/TiagoAnalyst/Professional_Portfolio/main/Booking%20system_dashboard/Data%20files/Recycling%20centre%20A%20-%20max.%20capacity.csv"
url3="https://raw.githubusercontent.com/TiagoAnalyst/Professional_Portfolio/main/Booking%20system_dashboard/Data%20files/Mapping-template-london-ward-map-2018.csv" #source: https://data.london.gov.uk/dataset/excel-mapping-template-for-london-boroughs-and-wards


# Attempt to fetch the CSV files using requests
try:
    response = requests.get(url, timeout=10) # timeout
    response2 = requests.get(url2, timeout=10) #timeout increased
    response3 = requests.get(url3, timeout=10)
    url_data = response.content.decode('utf-8')
    url2_data =  response2.content.decode('utf-8')
    url3_data = response3.content.decode('utf-8')

    #Read the CSV data into a DataFrame
    Main_dataset = pd.read_csv(StringIO(url_data))
    Capacity_dataset = pd.read_csv(StringIO(url2_data))
    wards_boroughs_source = pd.read_csv(StringIO(url3_data),usecols=['Ward name','Borough name'])

except requests.exceptions.RequestException as e:
    print(f"Error fetching the file: {e}")

In [51]:
wards_boroughs_source.head()

,Ward name,Borough name
0,Abbey,Barking and Dagenham
1,Alibon,Barking and Dagenham
2,Becontree,Barking and Dagenham
3,Chadwell Heath,Barking and Dagenham
4,Eastbrook,Barking and Dagenham


### Import of all datasets 

### Preview of datasets

In [ ]:
Main_dataset.head(5)

,Booking created,Postcode,Ward,Type of vehicle,Site,Are you hiring a vehicle?,Registration plate,Batteries (car & household),Books,Bulky non-recyclables,...,Tyres (maximum 2),Wooden furniture (whole or dismantled),Asbestos (needs to be double wrapped),Garden wood (decking/fencing/shed),Kitchen and bathroom fixtures and fixings,Plasterboard,Soil and turf,Wooden flooring (including laminate),Booking date/time,ClosureReason
0,2023-02-18 16:02:04,HA9 9SL,Barnhill,Car,Recycling centre A (Car / Small van),No,KEaakzIUFU,Yes,No,Yes,...,No,No,No,No,No,No,No,No,2023-04-07 09:00:00,Complete
1,2023-03-03 15:56:33,HA0 3TJ,Northwick Park,Car,Recycling centre A (Car / Small van),Yes,HIREVEHICLE,No,No,No,...,No,No,No,No,No,No,No,No,2023-04-02 10:00:00,Complete
2,2023-03-14 21:05:24,W3 6TW,East Acton,Car,Recycling centre A (Car / Small van),No,LAbzaaDZT,Yes,No,No,...,No,No,No,No,No,No,No,No,2023-04-03 13:30:00,No show
3,2023-03-20 13:42:29,W10 5QG,Kensal Green,Car,Recycling centre A (Car / Small van),No,LOaabzbzRKX,No,Yes,Yes,...,No,Yes,No,No,No,No,No,No,2023-04-02 13:00:00,Cancelled by customer
4,2023-03-20 14:10:47,HA9 6DH,NaN,Car,Recycling centre A (Car / Small van),No,FHbzbzhbtUMF,Yes,No,No,...,No,Yes,No,No,No,No,No,No,2023-04-02 14:00:00,No show


In [ ]:
Capacity_dataset.head()

,Date,Group,Total booked,Total spaces
0,01/04/2023,Recycling centre A (car / small van),145,400
1,01/04/2023,Recycling centre A (medium / large van),8,15
2,01/04/2023,Recycling centre A (pedestrian / bicycle),2,48
3,02/04/2023,Recycling centre A (car / small van),165,400
4,02/04/2023,Recycling centre A (medium / large van),14,15


In [ ]:
wards_boroughs_source.head()

,Ward name,Borough name
0,Abbey,Barking and Dagenham
1,Alibon,Barking and Dagenham
2,Becontree,Barking and Dagenham
3,Chadwell Heath,Barking and Dagenham
4,Eastbrook,Barking and Dagenham


## Data Cleaning

### Columns headers terminology check and amendments done

In [52]:
Capacity_dataset_amended = (
    Capacity_dataset
    .rename(columns={'Group':'Transport type'
            ,'Date':'Booking_date'})
    .assign(**{'Transport type': lambda c: c['Transport type'].apply(lambda c: c[c.find('(')+1:c.find(')')])})
    .assign(
        Booking_dayofweek = lambda z: pd.to_datetime(z['Booking_date'], format='%d/%m/%Y').dt.day_name(),
        Booking_month = lambda a: pd.to_datetime(a['Booking_date'], format='%d/%m/%Y').dt.month_name()
    )
)
Capacity_dataset_amended

,Booking_date,Transport type,Total booked,Total spaces,Booking_dayofweek,Booking_month
0,01/04/2023,car / small van,145,400,Saturday,April
1,01/04/2023,medium / large van,8,15,Saturday,April
2,01/04/2023,pedestrian / bicycle,2,48,Saturday,April
3,02/04/2023,car / small van,165,400,Sunday,April
4,02/04/2023,medium / large van,14,15,Sunday,April
...,...,...,...,...,...,...
772,30/03/2024,medium / large van,15,15,Saturday,March
773,30/03/2024,pedestrian / bicycle,0,48,Saturday,March
774,31/03/2024,car / small van,155,400,Sunday,March
775,31/03/2024,medium / large van,15,15,Sunday,March


In [53]:
#check the names of the west London Boroughs
West_London_boroughs=['Brent','Ealing','Harrow','Hillingdon','Hounslow','Richmond upon Thames']

West_London_wards=(
    wards_boroughs_source
    .loc[wards_boroughs_source['Borough name'].isin(West_London_boroughs)].reset_index(drop=True)
)
West_London_wards.loc[West_London_wards['Ward name'].duplicated(keep=False)]

,Ward name,Borough name
1,Barnhill,Brent
14,Queensbury,Brent
58,Queensbury,Harrow
65,Barnhill,Hillingdon


In [ ]:
#fixing the Ward name duplicates for Brent 

West_London_wards_amended= (
    West_London_wards
    .assign(
        **{
            'Ward name': lambda West_London_wards_: West_London_wards_['Ward name']
            .mask((West_London_wards_['Ward name']=='Barnhill') &(West_London_wards_['Borough name']=='Brent'),'Barnhill - Brent')
            .mask((West_London_wards_['Ward name']=='Queensbury') & (West_London_wards_['Borough name']=='Brent'),'Queensbury - Brent')
            }
        )
)

In [ ]:
Main_dataset_amended = (
    Main_dataset
    .rename(
        columns={
            'Mattresses (from your home only)':'Mattresses',
            'Clothes and textiles (mixed)':'clothes and textiles',
            'Site':'Transport type',
            'Ward':'Ward name'
        }
    )
    .assign(
        Booking_date = lambda x: pd.to_datetime(x['Booking date/time']).dt.strftime('%d/%m/%Y'),
        Booking_time = lambda y: pd.to_datetime(y['Booking date/time']).dt.strftime('%H:%M')
    )
    .drop(columns=['Booking date/time'])
    .assign(
        **{'Booking created': pd.to_datetime(Main_dataset['Booking created']).dt.date},
        Booking_dayofweek = lambda z: pd.to_datetime(z['Booking_date'], format='%d/%m/%Y').dt.day_name(),
        Booking_month = lambda a: pd.to_datetime(a['Booking_date'], format='%d/%m/%Y').dt.month_name(),
        Booking_year= lambda x: pd.to_datetime(x['Booking_date'], format='%d/%m/%Y').dt.year
    )
    .assign(
        **{
        'Transport type': lambda b: b['Transport type'].apply(lambda b: b[b.find('(')+1:b.find(')')]).str.lower()
        }
    )
    .assign(
        **{
            'Ward name': lambda wards_replace:wards_replace['Ward name']
            .mask(
                (wards_replace['Ward name']=='Barnhill') & 
                (wards_replace['Postcode'].str.contains('HA9 9')), #all postcodes starting with HA9 or NW9 assigned to Barnhill Brent -https://www.doogal.co.uk/AdministrativeAreas?ward=E05013497
                 'Barnhill - Brent'
            )
            .mask(
                (wards_replace['Ward name']=='Queensbury') & 
                (wards_replace['Postcode'].str.contains('HA8|NW9|HA7')), #all postcodes starting with HA9 or NW9 assigned to Queensbury Brent -  Brenthttps://www.doogal.co.uk/AdministrativeAreas?ward=E05013508
                 'Queensbury - Brent'
            )
        }
    )
    .merge(
        West_London_wards_amended[['Ward name','Borough name']],
        on=['Ward name'],
        how='left'
    )   
    .assign(
        **{
            'Borough name': lambda other_wards: other_wards['Borough name']
            .mask(
                   other_wards['Ward name'].notnull() &
                   (other_wards['Borough name'].isnull()),
                   'Other Boroughs'
                )
        }
    )
    .assign(
        **{
            'Ward name': lambda empty_cell: empty_cell['Ward name']
               .mask(
                   empty_cell['Ward name'].isnull(),
                   'No info'
               )
        }
    )
    .assign(
        **{
            'Borough name': lambda empty_cells: empty_cells['Borough name']
               .mask(
                   empty_cells['Borough name'].isnull(),
                   'No info'
               )
        }
    )
)

Main_dataset_amended


,Booking created,Postcode,Ward name,Type of vehicle,Transport type,Are you hiring a vehicle?,Registration plate,Batteries (car & household),Books,Bulky non-recyclables,...,Plasterboard,Soil and turf,Wooden flooring (including laminate),ClosureReason,Booking_date,Booking_time,Booking_dayofweek,Booking_month,Booking_year,Borough name
0,2023-02-18,HA9 9SL,Barnhill - Brent,Car,car / small van,No,KEaakzIUFU,Yes,No,Yes,...,No,No,No,Complete,07/04/2023,09:00,Friday,April,2023,Brent
1,2023-03-03,HA0 3TJ,Northwick Park,Car,car / small van,Yes,HIREVEHICLE,No,No,No,...,No,No,No,Complete,02/04/2023,10:00,Sunday,April,2023,Brent
2,2023-03-14,W3 6TW,East Acton,Car,car / small van,No,LAbzaaDZT,Yes,No,No,...,No,No,No,No show,03/04/2023,13:30,Monday,April,2023,Ealing
3,2023-03-20,W10 5QG,Kensal Green,Car,car / small van,No,LOaabzbzRKX,No,Yes,Yes,...,No,No,No,Cancelled by customer,02/04/2023,13:00,Sunday,April,2023,Brent
4,2023-03-20,HA9 6DH,No info,Car,car / small van,No,FHbzbzhbtUMF,Yes,No,No,...,No,No,No,No show,02/04/2023,14:00,Sunday,April,2023,No info
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52463,2024-03-31,NW6 6SH,No info,Car,car / small van,No,LAaabzbzXUJ,No,No,No,...,No,No,No,Complete,31/03/2024,15:30,Sunday,March,2024,No info
52464,2024-03-31,W10 2QS,No info,Car,car / small van,No,YHHty,No,No,No,...,No,No,No,Complete,31/03/2024,15:30,Sunday,March,2024,No info
52465,2024-03-31,HA0 4RH,Wembley Central,Car,car / small van,No,AYbzbzkzIYHZ,No,No,No,...,No,No,No,Complete,31/03/2024,15:30,Sunday,March,2024,Brent
52466,2024-03-31,NW6 6TH,Queens Park,Car,car / small van,No,KhbtWVH,No,No,No,...,No,No,No,Complete,31/03/2024,15:30,Sunday,March,2024,Brent


In [ ]:
app = dash.Dash(__name__)

#-----------------------------------------

Main_dataset_amended_details=Main_dataset_amended[
    [
        'Booking_date',
        'Booking_month',
        'Booking_year',
        'Booking_dayofweek',
        'Borough name',
        'Type of vehicle',
        'Transport type',
        'Registration plate',
        'ClosureReason',
        'Ward name'
    ]
]

Capacity_dataset_amended

#--------------------------------------------
# Web app layout

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
        dbc.Row([
            dbc.Col([
                    html.H1("Booking system dashboard - Financial year 2023-2024", style={'text-align':'center'}),
                    html.Br()
                ], width=12
                ) 
        ]),
        dbc.Row([
            dbc.Col([
                    html.Div('Please choose a month:'),
                    dcc.Dropdown(id='slct_month',
                                options=[x for x in Main_dataset_amended_details['Booking_month'].unique()],
                                multi=False,
                                value= "April"),
                    html.Div(id='output_month',children=[])
                ],width=4
            )
        ]), 
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='chart2',
                            figure={}
                            )
            ], width=6),
            dbc.Col([
                dcc.Graph(id='chart3',
                            figure={}
                            )
            ], width=6)
        ]),
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='chart4',
                            figure={} 
                            )
            ], width=6)
        ]),
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='chart1',
                            figure={} 
                            )
            ], width=12)
        ]),
    ]
)

    
    #https://youtu.be/1nEL0S8i2Wk?si=hCCh51-MG2zYgHVi

#--------------------------------------------------------------------
#Connect the Plotly graphs with Dash Components -- callback
@app.callback(
    [
        Output(component_id='output_month',component_property='children'),
        Output(component_id='chart1',component_property='figure'),
        Output(component_id='chart2',component_property='figure'),
        Output(component_id='chart3',component_property='figure'),
        Output(component_id='chart4',component_property='figure')
    ],
    [
        Input(component_id='slct_month', component_property='value')
    ]
)

def update_graphs(slct_month): 
    print(type(slct_month))

    #container

    container_month = "The month chosen by user was: {}".format(slct_month)

# VIDEO A SEGUIR!


    Booking_capacity = Capacity_dataset_amended.copy()
    Booking_capacity = Booking_capacity[Booking_capacity['Booking_month'] == slct_month]

    # Ensure 'Booking_date' is in datetime format and sort by 'Booking_date'

    Booking_capacity['Booking_date'] = pd.to_datetime(Booking_capacity['Booking_date'],format='%d/%m/%Y', dayfirst=True)

    #Group total spaces and total bookings data by Booking_date

    dff_capacity_grouped = Booking_capacity.groupby('Booking_date').sum('Total spaces')
    dff_total_bookings = Booking_capacity.groupby('Booking_date').sum('Total booked')
    
    #Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces

    fig.add_trace(
        go.Bar(
            x=dff_total_bookings.index,
            y=dff_total_bookings['Total booked'],
            name='No. of bookings'
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            x=dff_capacity_grouped.index,
            y=dff_capacity_grouped['Total spaces'],
            name='Daily capacity',
            ),
        secondary_y=True
    )

    #Add figure title
    fig.update_layout(
        title_text="Number of bookings made vs bookings capacity"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Date")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Bookings capacity</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>No. of bookings</b>", secondary_y=True)
    fig.update_yaxes(range = [0,500])
    fig.update_xaxes(
        tickformat='%d',
        tickmode='array',
        tickvals=Booking_capacity['Booking_date']

    )
    fig.show()

    # Chart2
    
    filtered_dataset_by_month = Main_dataset_amended_details.loc[Main_dataset_amended_details['Booking_month'] == slct_month]
    
    #Get the value counts of "ClosureReason"

    bookings_classification = filtered_dataset_by_month['ClosureReason'].value_counts().reset_index()
    bookings_classification.columns=['ClosureReason','Count']
    
        #Create the pie
    
    fig2 =px.pie(bookings_classification, names = 'ClosureReason',values='Count',title='Closure Reason distribution')
    fig2.show()

    #Create the gauge chart
    Gauge_chart = (Booking_capacity['Total booked'].sum()/Booking_capacity['Total spaces'].sum())*100


    fig3 = go.Figure(
            go.Indicator(
                mode= "gauge+number",
                value=Gauge_chart,
                number = {'suffix':"%"},
                domain={'x':[0,1],'y':[0,1]},
                title ={'text':"Bookings capacity usage"},
                gauge ={'axis': {'range': [0,100]},
                        'bar':{'color':'grey'},
                        'steps':[
                            {'range':[0,80],'color':"lightgreen"},
                            {'range':[80,90],'color':"yellow"},
                            {'range':[90,100],'color':"red"}
                            ]
                }
            )
        )
    fig3.show()

    #chart 4 - busiest weekdays 
        #data  manipulation

    Ordered_weekday = ['Monday','Thursday','Friday','Saturday','Sunday']
    Bookings_dayofweek = Booking_capacity['Booking_dayofweek'].unique()

    Bookings_dayofweek_table=[]

    for i in Bookings_dayofweek:
        var_temp = Booking_capacity.loc[Booking_capacity['Booking_dayofweek']==i].copy()
        var_temp2 = var_temp['Total booked'].sum()/var_temp['Booking_date'].nunique()
        Bookings_dayofweek_table.append({'Booking_dayofweek':i,'Average no. of bookings': var_temp2})

    Bookings_dayofweek_table_df = pd.DataFrame(Bookings_dayofweek_table)
    Bookings_dayofweek_table_df = Bookings_dayofweek_table_df.set_index('Booking_dayofweek').reindex(Ordered_weekday).reset_index()

        #determine the day with the highest average
    max_avg_booking_day = Bookings_dayofweek_table_df.loc[Bookings_dayofweek_table_df['Average no. of bookings'].idxmax(),'Booking_dayofweek']

    #colours

    colors=['blue']*len(Bookings_dayofweek_table_df)
    max_day_index = Bookings_dayofweek_table_df.index[Bookings_dayofweek_table_df['Booking_dayofweek']== max_avg_booking_day].tolist()[0]
    colors[max_day_index] = 'crimson'

    fig4 = go.Figure(
        go.Bar(
                x=Bookings_dayofweek_table_df['Booking_dayofweek'],
                y=Bookings_dayofweek_table_df['Average no. of bookings'],
                marker_color = colors
        )
    )

    fig4.update_layout(title_text='Average no. of bookings per day of week')
    
    return container_month, fig, fig2, fig3, fig4
#--------------------------------------------------------------------

if __name__=='__main__':
    app.run_server(debug=True)


##Web app layout 



#Change dashboard features - https://plotly.com/python/v3/create-online-dashboard-legacy/

# my_dboard = dashboard.Dashboard()

# box_a = {
#     'type':'box',
#     'boxType':'plot',
#     'fileId': fig,
#     'shareKey': None,
#     'title':'YYYYYY'
# }

# text_for_box = """

# ## Distributions:

# #### Scatter Plot
# 1. Ranging 0 - 500
# 2. Even distribution

# #### Box Plot
# 1. Similar range
# 2. Outliers present in trace 1 and trace 3

# You can view more markdown tips [here] - A TUA MAE
# """

# box_b ={
#     'type':'box',
#     'boxType':'text',
#     'fileId':fig2,
#     'text': text_for_box,
#     'title':'XXXXXXX'
# }

# box_c ={
#     'type':'box',
#     'boxType':'plot' ,
#     'title': 'ZZZZZZZ'
# }

# my_dboard.insert(box_c)
# my_dboard.insert(box_a,'above',1)
# my_dboard.insert(box_b, 'left', 1, fill_percent=40)

# my_dboard.get_box(1)
# my_dboard.get_box(1)['title'] = 'test123'

# my_dboard['settings']['title'] = 'Booking system dashboard with Python' # title for the dashboard
# my_dboard['settings']['foregroundColor'] = '#000000'



# my_dboard.get_preview()

#trocar plots - my_dboard.swap(2,3)

<class 'str'>


<class 'str'>


<class 'str'>


<class 'str'>
